# Talktorial 1

# Compound data acquisition (ChEMBL)

#### Developed in the CADD seminars 2017 and 2018, AG Volkamer, Charité/FU Berlin 

Paula Junge and Svetlana Leng - adapted by Gautier Peyrat

## Aim of this talktorial

We learn how to extract data from ChEMBL:

* Find ligands which were tested on a certain target
* Filter by available bioactivity data
* Calculate pIC50 values
* Merge dataframes and draw extracted molecules

## Learning goals


### Theory

* ChEMBL database
    * ChEMBL web services
    * ChEMBL webresource client
* Compound activity measures
    * IC50
    * pIC50

### Practical
    
Goal: Get list of compounds with bioactivity data for a given target

* Connect to ChEMBL database
* Get target data (EGFR kinase)
* Bioactivity data
    * Download and filter bioactivities
    * Clean and convert
* Compound data
    * Get list of compounds
    * Prepare output data
* Output
    * Draw molecules with highest pIC50
    * Write output file


## References

* ChEMBL bioactivity database (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5210557/)
* ChEMBL web services: <i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620 (https://academic.oup.com/nar/article/43/W1/W612/2467881) 
* ChEMBL webrescource client GitHub (https://github.com/chembl/chembl_webresource_client)
* myChEMBL webservices version 2.x (https://github.com/chembl/mychembl/blob/master/ipython_notebooks/09_myChEMBL_web_services.ipynb)
* ChEMBL web-interface (https://www.ebi.ac.uk/chembl/)
* EBI-RDF platform (https://www.ncbi.nlm.nih.gov/pubmed/24413672)
* IC50 and pIC50 (https://en.wikipedia.org/wiki/IC50)
* UniProt website (https://www.uniprot.org/)

_____________________________________________________________________________________________________________________


## Theory

### ChEMBL database

* Open large-scale bioactivity database
* **Current data content (as of 10.2018):**
    * \>1.8 million distinct compound structures
    * \>15 million activity values from 1 million assays
    * Assays are mapped to ∼12 000 targets
* **Data sources** include scientific literature, PubChem bioassays, Drugs for Neglected Diseases Initiative (DNDi), BindingDB database, ...
* ChEMBL data can be accessed via a [web-interface](https://www.ebi.ac.uk/chembl/), the [EBI-RDF platform](https://www.ncbi.nlm.nih.gov/pubmed/24413672) and the [ChEMBL web services](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4489243/#B5)
 
    
#### ChEMBL web services

* RESTful web service
* ChEMBL web service version 2.x resource schema: 

[![ChEMBL web service schema](images/chembl_webservices_schema_diagram.jpg)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4489243/figure/F2/)

*Figure 1:* 
"ChEMBL web service schema diagram. The oval shapes represent ChEMBL web service resources and the line between two resources indicates that they share a common attribute. The arrow direction shows where the primary information about a resource type can be found. A dashed line indicates the relationship between two resources behaves differently. For example, the `Image` resource provides a graphical based representation of a `Molecule`."
Figure and description taken from: [<i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620](https://academic.oup.com/nar/article/43/W1/W612/2467881).


#### ChEMBL webresource client

* Python client library for accessing ChEMBL data
* Handles interaction with the HTTPS protocol
* Lazy evaluation of results -> reduced number of network requests

### Compound activity measures

#### IC50 

* [Half maximal inhibitory concentration](https://en.wikipedia.org/wiki/IC50)
* Indicates how much of a particular drug or other substance is needed to inhibit a given biological process by half

<img src="https://s3.amazonaws.com/cdn.graphpad.com/faq/1356/images/1356b.png" width="450" align="center" >

*Figure 2:* Visual demonstration of how to derive an IC50 value: Observe binding (or function) on vertical axis and concentration on horizontal axis; then identify max and min inhibition; IC50 is the concentration at which the curve passes through the 50% inhibition level.

#### pIC50

* To facilitate the comparison of IC50 values, we define pIC50 values on a logarithmic scale, such that <br />
    $ pIC_{50} = -log_{10}(IC_{50}) $ where $ IC_{50}$ is specified in units of M.
* Higher pIC50 values indicate exponentially greater potency of the drug
* pIC50 is given in terms of molar concentration (mol/L or M) <br />
    * IC50 should be specified in M to convert to pIC50  
    * For nM: $pIC_{50} = -log_{10}(IC_{50}*10^{-9})= 9-log_{10}(IC_{50}) $
    
Besides, IC50 and pIC50, other bioactivity measures are used, such as the equilibrium constant [KI](https://en.wikipedia.org/wiki/Equilibrium_constant) and the half maximal effective concentration  [EC50](https://en.wikipedia.org/wiki/EC50).

## Practical

In the following, we want to download all molecules that have been tested against our target of interest, the EGFR kinase.

### Connect to ChEMBL database

First, the ChEMBL webresource client as well as other python libraries are imported.

In [1]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import math
from rdkit.Chem import PandasTools
import os
import pickle
import sys
from nbautoeval import run_yaml_quiz

sys.path.insert(1, f'../corrections/exercices')

from T1_ChEMBL import exo_convert_nM
from T1_ChEMBL import exo_calc_pIC50

Create resource objects for API access.

In [2]:
targets = new_client.target
compounds = new_client.molecule
bioactivities = new_client.activity

## Target data

* Get UniProt-ID (http://www.uniprot.org/uniprot/P00533) of the target of interest (EGFR kinase) from UniProt website (https://www.uniprot.org/)
* Use UniProt-ID to get target information
* Select a different UniProt-ID if you are interested into another target

In [3]:
# define uniprot_id as "P00533"
uniprot_id = "P00533"
# Get target information from ChEMBL but restrict to specified values only
target_P00533 = targets.get(target_components__accession=uniprot_id) \
                       .only('target_chembl_id', 'organism', 'pref_name', 'target_type')
print(type(target_P00533))
pd.DataFrame.from_records(target_P00533)

<class 'chembl_webresource_client.query_set.QuerySet'>


,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,Epidermal growth factor receptor erbB1,CHEMBL203,SINGLE PROTEIN
1,Homo sapiens,Epidermal growth factor receptor erbB1,CHEMBL203,SINGLE PROTEIN
2,Homo sapiens,Epidermal growth factor receptor and ErbB2 (HE...,CHEMBL2111431,PROTEIN FAMILY
3,Homo sapiens,Epidermal growth factor receptor,CHEMBL2363049,PROTEIN FAMILY
4,Homo sapiens,MER intracellular domain/EGFR extracellular do...,CHEMBL3137284,CHIMERIC PROTEIN


### After checking the entries, we select the first entry as our target of interest
`CHEMBL203`: It is a single protein and represents the human Epidermal growth factor receptor (EGFR, also named erbB1) 

In [4]:
# select the first row of the dataframe "target_P00533"
target = target_P00533[0]
# Check your selection by displaying the content of "target"
target

{'organism': 'Homo sapiens',
 'pref_name': 'Epidermal growth factor receptor erbB1',
 'target_chembl_id': 'CHEMBL203',
 'target_type': 'SINGLE PROTEIN'}

Save selected ChEMBL-ID.

In [5]:
# Select only the chembl_id on a variable called "chembl_id" and display it
chembl_id = target['target_chembl_id']
chembl_id

'CHEMBL203'

### Bioactivity data

Now, we want to query bioactivity data for the target of interest.

#### Download and filter bioactivities for the target

In this step, we download and filter the bioactivity data and only consider:

* human proteins
* bioactivity type IC50
* exact measurements (relation '=')    
* binding data (assay type 'B')

In [6]:
# Complete the following instructions to execute the correct query:
# bioact = bioactivities.filter(target_chembl_id = chembl_id).filter(type = 'IC50' ) \
#                       .filter(relation = '=' ) \
#                       .filter(assay_type = 'B') \
#                       .only('activity_id','assay_chembl_id', 'assay_description', 'assay_type', \
#                             'molecule_chembl_id', 'type', 'units', 'relation', 'value', \
#                             'target_chembl_id', 'target_organism')
# len(bioact), len(bioact[0]), type(bioact), type(bioact[0])

In [8]:
#type(bioact)

If you experience difficulties to query the ChEMBL database, we provide here a file containing the results for the query in the previous cell (11 April 2019). We do this using the Python package pickle which serializes Python objects so they can be saved to a file, and loaded in a program again later on.
(Learn more about object serialization on [DataCamp](https://www.datacamp.com/community/tutorials/pickle-python-tutorial))

You can load the "pickled" compounds by uncommenting and running the next cell.

In [9]:
import pickle
bioact = pickle.load(open("../data/T1/EGFR_compounds_from_chembl_query_20190411.p", "rb"))

In [10]:
len(bioact), len(bioact[0]), type(bioact), type(bioact[0])

(6262, 11, list, dict)

#### Clean and convert bioactivity data

The data is stored as a list of dictionaries

In [11]:
# Display the first element of the queryset
bioact[0]

{'activity_id': 32260,
 'assay_chembl_id': 'CHEMBL674637',
 'assay_description': 'Inhibitory activity towards tyrosine phosphorylation for the epidermal growth factor-receptor kinase',
 'assay_type': 'B',
 'molecule_chembl_id': 'CHEMBL68920',
 'relation': '=',
 'target_chembl_id': 'CHEMBL203',
 'target_organism': 'Homo sapiens',
 'type': 'IC50',
 'units': 'uM',
 'value': '0.041'}

Convert to pandas dataframe (this might take some minutes).

In [12]:
# convert the queryset to a dataframe and store it as "bioact_df" (we already used the required method)
bioact_df = pd.DataFrame.from_records(bioact)
#then display the 10 first rows by using {dataframe_name}.head(10)
bioact_df.head(10)

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,target_chembl_id,target_organism,type,units,value
0,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,uM,0.041
1,32267,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,CHEMBL69960,=,CHEMBL203,Homo sapiens,IC50,uM,0.17
2,32680,CHEMBL677833,In vitro inhibition of Epidermal growth factor...,B,CHEMBL137635,=,CHEMBL203,Homo sapiens,IC50,uM,9.3
3,32770,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL306988,=,CHEMBL203,Homo sapiens,IC50,uM,500
4,32772,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL66879,=,CHEMBL203,Homo sapiens,IC50,uM,3000
5,32780,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL77085,=,CHEMBL203,Homo sapiens,IC50,uM,96
6,33406,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,CHEMBL443268,=,CHEMBL203,Homo sapiens,IC50,uM,5.31
7,34039,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL76979,=,CHEMBL203,Homo sapiens,IC50,uM,264
8,34041,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL76589,=,CHEMBL203,Homo sapiens,IC50,uM,0.125
9,34049,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL76904,=,CHEMBL203,Homo sapiens,IC50,uM,35


In [13]:
bioact_df.shape

(6262, 11)

Delete entries with missing values.

In [14]:
bioact_df = bioact_df.dropna(axis=0, how = 'any')
bioact_df.shape

(6261, 11)

Delete duplicates:
Sometimes the same molecule (`molecule_chembl_id`) has been tested more than once, in this case, we only keep the first one.

In [15]:
# Use the "drop_duplicates" method on 'molecule_chembl_id' column and use the correct argument to keep
# only the first of duplicated rows.
bioact_df = bioact_df.drop_duplicates('molecule_chembl_id', keep='first')
bioact_df.shape

(5041, 11)

We would like to only keep bioactivity data measured in molar units. The following print statements will help us to see what units are contained and to control what is kept after dropping some rows.

In [16]:
print(bioact_df.units.unique())
bioact_df = bioact_df.drop(bioact_df.index[~bioact_df.units.str.contains('M')])
print(bioact_df.units.unique())
bioact_df.shape

['uM' 'nM' 'M' "10'1 ug/ml" 'ug ml-1' "10'-1microM" "10'1 uM"
 "10'-1 ug/ml" "10'-2 ug/ml" "10'2 uM" '/uM' "10'-6g/ml" 'mM' 'umol/L'
 'nmol/L']
['uM' 'nM' 'M' "10'-1microM" "10'1 uM" "10'2 uM" '/uM' 'mM']


(4972, 11)

Since we deleted some rows, but we want to iterate over the index later, we reset index to be continuous.

In [17]:
bioact_df = bioact_df.reset_index(drop=True) 
bioact_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,target_chembl_id,target_organism,type,units,value
0,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,uM,0.041
1,32267,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,CHEMBL69960,=,CHEMBL203,Homo sapiens,IC50,uM,0.17
2,32680,CHEMBL677833,In vitro inhibition of Epidermal growth factor...,B,CHEMBL137635,=,CHEMBL203,Homo sapiens,IC50,uM,9.3
3,32770,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL306988,=,CHEMBL203,Homo sapiens,IC50,uM,500
4,32772,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CHEMBL66879,=,CHEMBL203,Homo sapiens,IC50,uM,3000


To allow further comparison of the IC50 values, we convert all units to nM.

In [18]:
exo_convert_nM.example()

GridBox(children=(HTML(value='<span style="font-size:medium;"\'>arguments</span>', _dom_classes=('header',)), …

In [19]:
# ['uM' 'nM' 'M' "10'-1microM" "10'1 uM" "10'2 uM" '/uM' 'mM']
def convert_nM(df):
    # create an empty list "bioactivity_nM"
    bioactivity_nM = []
    # for i, row in df[['units', 'value']].iterrows():
    for i, row in df[['units', 'value']].iterrows():
        unit = row['units']
        bioactivity = row['value']
        # convert the bioactivity in nanomolar unit, be careful with all the different unit expressions
        # If the unit is "nM", the function will return directly the bioactivity value without converting it.
        # append the converted bioactivity to the list
        if unit != "nM":
            if unit == "pM":
                value = float(bioactivity) / 1000
            elif unit == "10'-11M":
                value = float(bioactivity) / 100
            elif unit == "10'-10M":
                value = float(bioactivity) / 10
            elif unit == "10'-8M":
                value = float(bioactivity) * 10
            elif unit == "10'-1microM" or unit == "10'-7M":
                value = float(bioactivity) * 100
            elif unit == "uM" or unit == "/uM" or unit == "10'-6M":
                value = float(bioactivity) * 1000
            elif unit == "10'1 uM":
                value = float(bioactivity) * 10000
            elif unit == "10'2 uM":
                value = float(bioactivity) * 100000
            elif unit == "mM":
                value = float(bioactivity) * 1000000
            elif unit == "M":
                value = float(bioactivity) * 1000000000
            else:
                print('unit not recognized...', unit)
            bioactivity_nM.append(value)
        else:
            bioactivity_nM.append(bioactivity)

    # Create a new colum "value" in the dataframe by using the list of bioactivites
    df['value'] = bioactivity_nM
    # Create another column "units" with the correct unit symbol
    df['units'] = 'nM'
    return df

In [20]:
exo_convert_nM.correction(convert_nM)

GridBox(children=(HTML(value='<span style="font-size:medium;"\'>arguments</span>', _dom_classes=('header', '')…

In [21]:
bioact_df = convert_nM(bioact_df)
# Display the first rows of the dataframe

### Compound data

We have a data frame containing all molecules tested (with the respective measure) against EGFR. Now, we want to get the molecules that are stored behind the respective ChEMBL IDs. 

#### Get list of compounds

Let's have a look at the compounds from ChEMBL we have defined bioactivity data for. First, we retrieve ChEMBL ID and structures for the compounds with desired bioactivity data.

In [ ]:
# Get all the molecule chembl ids from bioac_df in a list 'cmpd_id_list'

# Query the chembl by using "compounds" table with this filter : filter(molecule_chembl_id__in = cmpd_id_list)
# and select only "molecule_chembl_id" and "molecules_structures" fields, store the result in "compound_list"
# variable.


Then, we convert the list to a pandas dataframe and delete duplicates (again, the pandas function might take some time).

In [ ]:
# create a dataframe called "compound_df" from the query "compound_list"

# drop the duplicates and keep only the first when there is several "molecule_chembl_id"

print(compound_df.shape)
print(bioact_df.shape)
compound_df.head()

So far, we have multiple different molecular structure representations. We only want to keep the canonical SMILES.

In [ ]:
for i, cmpd in compound_df.iterrows():
    if compound_df.loc[i]['molecule_structures'] != None:
        compound_df.loc[i]['molecule_structures'] = cmpd['molecule_structures']['canonical_smiles']

print(compound_df.shape)

#### Prepare output data

Merge values of interest in one dataframe on ChEMBL-IDs:
* ChEMBL-IDs
* SMILES
* units
* IC50

In [ ]:
output_df = pd.merge(bioact_df[['molecule_chembl_id','units','value']], compound_df, on='molecule_chembl_id')
print(output_df.shape)
output_df.head()

For distinct column names, we rename IC50 and SMILES columns.

In [ ]:
# rename columns in output_df dataframe : 'molecule_structures' --> 'smiles' and 'value' --> 'IC50'

output_df.shape

If we do not have a SMILES representation of a compound, we can not further use it in the following talktorials. Therefore, we delete compounds without SMILES column.

In [ ]:
output_df = output_df[~output_df['smiles'].isnull()]
print(output_df.shape)
output_df.head()

In the next cell, you see that the low IC50 values are difficult to read. Therefore, we prefer to convert the IC50 values to pIC50.

In [ ]:
output_df = output_df.reset_index(drop=True)
ic50 = output_df.IC50.astype(float) 
print(len(ic50))
print(ic50.head(10))

In [ ]:
# create a function to convert IC50 to pIC50, it takes IC50 in argument and return pIC50.
# IC50 comes as a string, it must be converted to float to calculate pIC50
# pIC50=-log10(IC50 mol/l)
# for nM: -log10(IC50*10**-9)= 9-log10(IC50)
# in the output df, a column called 'pIC50' contains all pIC50.


In [ ]:
exo_calc_pIC50.example()

In [ ]:
def calc_pIC50(df):
    return df

In [ ]:
exo_calc_pIC50.correction(calc_pIC50)

In [ ]:
output_df = calc_pIC50(output_df) # apply the function on the dataframe
output_df.head()

### Collected bioactivity data for EGFR

Let's have a look at our collected data set.
#### Draw molecules
In the next steps, we add a molecule column to our datafame and look at the structures of the molecules with the highest pIC50 values. 

In [ ]:
PandasTools.AddMoleculeColumnToFrame(output_df, smilesCol='smiles')

Sort molecules by pIC50.

In [ ]:
output_df.sort_values(by="pIC50", ascending=False, inplace=True)
output_df.reset_index(drop=True, inplace=True)

Show the most active molecules = molecules with the highest pIC50 values.

In [ ]:
output_df.drop("smiles", axis=1).head()

In [ ]:
len(output_df)

#### Write output file
To use the data for the following talktorials, we save the data as csv file. Note that it is advisable to drop the molecule column (only contains an image of the molecules) when saving the data.

In [ ]:
output_df.drop("ROMol", axis=1).to_csv("../data/T1/EGFR_compounds.csv")

## Discussion

In this tutorial, we collected all available bioactivity data for our target of interest from the ChEMBL database. We filtered the data set to only contain molecules with measured IC50 or pIC50 bioactivity values. 

Be aware that ChEMBL data originates from various sources. Compound data has been generated in different labs by different people all over the world. Therefore, we have to be cautious with the predictions we make using this dataset. It is always important to consider the source of the data and consistency of data production assays when interpreting the results and determining how much confidence we have in our predictions.

In the next tutorials we will filter our acquired data by the Lipinski rule of five and by unwanted substructures. Another important step would be to clean the data and remove duplicates. As this is not shown in any of our talktorials (yet), we would like to refer to the standardiser library ([github Francis Atkinson](https://github.com/flatkinson/standardiser)) or [MolVS](https://molvs.readthedocs.io/en/latest/) as possible tools for this task.

# Quiz

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T1_ChEMBL.yaml", "theoric-quiz")

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T1_ChEMBL.yaml", "code-quiz")